In [1]:
!pip install mido

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 2.1 MB/s eta 0:00:00


In [2]:
# Imports

import os
import csv
import glob
import mido
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm

In [3]:
def generate_target_array(targets, chunk_size, chromatic=True):

	if chromatic:
		n_midi_notes = 12
	else:
		n_midi_notes   = 108

	song_len = int(chunk_size * np.ceil(len(targets)/chunk_size))
	target_array = np.zeros((song_len, n_midi_notes))

	for tick, target_chord in enumerate(targets):
		for target_note in [int(note)-1 for note in target_chord if note != '']:
			note = target_note % 12 if chromatic else target_note
			target_array[tick, note] = 1

	return target_array


def add_noise_to_target(target_array, noise=0.5):

	sample_array = target_array + 0.5 * np.random.randn(*target_array.shape)

	return(sample_array)


def get_songs(songs_path, chunk_size, chromatic=True, noise=0.5):

	n_pad_song_end = 20

	files = dict()
	for filepath in glob.glob(os.path.join(songs_path, '*.csv')):
		file = os.path.split(filepath)[-1]
		if '_' in file:
			opera = file.split('_')[0]
		else:
			opera = file.replace('.csv', '')
		files[opera] = glob.glob(os.path.join(songs_path, f'{opera}*.csv'))

	songs_dict = dict()
	for opera in files:
		songs_dict[opera] = dict()
		targets = []
		for file in files[opera]:
			with open(file, 'r') as f:
				targets += [el for el in list(csv.reader(f)) if el != []]
				targets += [[]] * n_pad_song_end

		target_array = generate_target_array(targets, chunk_size, chromatic)
		sample_array = add_noise_to_target(target_array, noise)
		songs_dict[opera]['target'] = target_array
		songs_dict[opera]['sample'] = sample_array

	return songs_dict


class Chunker():

	def __init__(self, songs_path, batch_size, chunk_size, chromatic=True, noise=0.5):
		self.batch_size = batch_size
		self.songs_dict = get_songs(songs_path, chunk_size, chromatic, noise)
		self.chunk_size = chunk_size
		self.song_pool  = list(self.songs_dict.keys())
		self.song_list = [np.random.choice(self.song_pool) for _ in range(batch_size)]
		self.t0  = [0 for _ in range(batch_size)]

	def create_chunck(self):
		n_dims = self.songs_dict[self.song_pool[0]]['target'].shape[1]
		target = np.zeros((self.batch_size, self.chunk_size, n_dims))
		sample = np.zeros((self.batch_size, self.chunk_size, n_dims))

		for n in range(self.batch_size):
			t0, t1 = self.t0[n], self.t0[n] + self.chunk_size
			target[n, :, :] = self.songs_dict[self.song_list[n]]['target'][t0:t1]
			sample[n, :, :] = self.songs_dict[self.song_list[n]]['sample'][t0:t1]
			if t1 > self.songs_dict[self.song_list[n]]['target'].shape[1]-self.chunk_size:
				self.song_list[n] = np.random.choice(self.song_pool)
				self.t0[n] = 0
			else:
				self.t0[n] += self.chunk_size

		return target, sample


chunk_size = 512
batch_size = 16
chromatic  = True
noise = 0.5

chunker = Chunker('./train', chunk_size, batch_size, chromatic, noise)
target, sample = chunker.create_chunck()
print(target.shape)
print(sample.shape)

(512, 16, 12)
(512, 16, 12)


In [4]:
# Save the model weights in this directory
weight_dir = "model_wts"

if not os.path.exists(weight_dir):
  os.mkdir(weight_dir)

In [16]:
# Functions to generate the sample from the CSV file
chunk_size = 512
batch_size = 16
chromatic  = True
noise = 0.5
num_chunks = 1000

chunker = Chunker('./train', chunk_size, batch_size, chromatic, noise)

In [17]:
# Generate and concatenate chunks
targets = []
samples = []

for _ in range(num_chunks):
    target, sample = chunker.create_chunck()
    targets.append(torch.tensor(target, dtype=torch.float32))
    samples.append(torch.tensor(sample, dtype=torch.float32))

# Concatenate all targets and samples
targets_tensor = torch.cat(targets)
samples_tensor = torch.cat(samples)


In [18]:
from torch.utils.data import Dataset, DataLoader

class ChunkTensorDataset(Dataset):
    def __init__(self, targets_tensor, samples_tensor):
        self.targets_tensor = targets_tensor
        self.samples_tensor = samples_tensor

    def __len__(self):
        return len(self.targets_tensor)

    def __getitem__(self, idx):
        return self.targets_tensor[idx], self.samples_tensor[idx]

# Create Dataset and DataLoader
dataset = ChunkTensorDataset(targets_tensor, samples_tensor)
train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [14]:
# Hyperparameters
input_size = 12
num_layers = 2
hidden_size = 64
learning_rate = 0.001
num_epochs = 2

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [19]:
class SimpleRNN(nn.Module):
    def __init__(self, input_size, num_layers, hidden_size):
        super(SimpleRNN, self).__init__()
        self.input_size = input_size
        self.num_layers = num_layers
        self.hidden_size = hidden_size

        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, input_size)
        self.sig = nn.Sigmoid()

    def forward(self, x, hidden=None):
        if hidden is None:
          batch_size = x.size(0)
          hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)

        out, hidden = self.rnn(x, hidden)
        out = self.fc1(out)
        out = self.sig(out)
        return out, hidden

model = SimpleRNN(input_size, num_layers, hidden_size).to(device=device)

# Loss and optimizer
criterion  = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    epoch_loss = 0
    for ground_truth, input_data in tqdm(train_dataloader):
        ground_truth = ground_truth.to(device=device)
        input_data = input_data.to(device=device)

        output, hidden = model(input_data)
        loss = criterion(output, ground_truth)
        current_loss = loss

        # Learning
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss / len(train_dataloader)}")


# Save the weights
weights_file = os.path.join(weight_dir, 'SimpleRNNweights.pth')
torch.save(model.state_dict(), weights_file)
print(f"Simple RNN weights saved to {dir}")

100%|██████████| 32000/32000 [03:20<00:00, 159.76it/s]


Epoch [1/2], Loss: 0.008837007218610372


100%|██████████| 32000/32000 [03:20<00:00, 159.49it/s]

Epoch [2/2], Loss: 6.000567968694243e-05
Simple RNN weights saved to <built-in function dir>


In [20]:
class SimpleLSTM(nn.Module):
    def __init__(self, input_size, num_layers, hidden_size):
        super(SimpleLSTM, self).__init__()
        self.input_size = input_size
        self.num_layers = num_layers
        self.hidden_size = hidden_size

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, input_size)
        self.sig = nn.Sigmoid()

    def forward(self, x, hidden=None):
        if hidden is None:
          batch_size = x.size(0)
          hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
          c0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)

        out, hidden = self.lstm(x, (hidden, c0))
        out = self.fc1(out)
        out = self.sig(out)
        return out, hidden

model = SimpleLSTM(input_size, num_layers, hidden_size).to(device=device)

# Loss and optimizer
criterion  = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    epoch_loss = 0
    for ground_truth, input_data in tqdm(train_dataloader):
        ground_truth = ground_truth.to(device=device)
        input_data = input_data.to(device=device)

        output, hidden = model(input_data)
        loss = criterion(output, ground_truth)
        current_loss = loss

        # Learning
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss / len(train_dataloader)}")

# Save the weights
weights_file = os.path.join(weight_dir, 'LSTMweights.pth')
torch.save(model.state_dict(), weights_file)
print(f"LSTM Model weights saved to {dir}")

100%|██████████| 32000/32000 [04:53<00:00, 108.91it/s]


Epoch [1/2], Loss: 0.016277532574986486


100%|██████████| 32000/32000 [04:53<00:00, 109.11it/s]

Epoch [2/2], Loss: 4.500220838267782e-05
LSTM Model weights saved to <built-in function dir>


In [21]:
class GRUModel(nn.Module):
    def __init__(self, input_size, num_layers, hidden_size):
        super(GRUModel, self).__init__()
        self.input_size = input_size
        self.num_layers = num_layers
        self.hidden_size = hidden_size

        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, input_size)
        self.sig = nn.Sigmoid()

    def forward(self, x, hidden=None):
        if hidden is None:
            batch_size = x.size(0)
            hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)

        out, hidden = self.gru(x, hidden)
        out = self.fc1(out)
        out = self.sig(out)
        return out, hidden

model = GRUModel(input_size, num_layers, hidden_size).to(device=device)


# Loss and optimizer
criterion  = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    epoch_loss = 0
    for ground_truth, input_data in tqdm(train_dataloader):
        ground_truth = ground_truth.to(device=device)
        input_data = input_data.to(device=device)

        output, hidden = model(input_data)
        loss = criterion(output, ground_truth)
        current_loss = loss

        # Learning
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss / len(train_dataloader)}")


# Save the weights
weights_file = os.path.join(weight_dir, 'GRUweights.pth')
torch.save(model.state_dict(), weights_file)
print(f"GRU Model weights saved to {dir}")

100%|██████████| 32000/32000 [06:54<00:00, 77.29it/s]


Epoch [1/2], Loss: 0.008077771728630728


100%|██████████| 32000/32000 [06:50<00:00, 77.93it/s]

Epoch [2/2], Loss: 2.6649561561070332e-08
GRU Model weights saved to <built-in function dir>
